<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=ee1881efda880ea3402504f656d7ea69c69b7ff2dbaa67c3bee01948de67cf46
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-01 12:41:22
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  1.57 L
Current:  1.40 C
-------------------
Today PnL: 83.33 K (0.6%)
Current PnL: -21.98 L (-14.59%)
CY Booked + Current PnL: -10.54 L (-7.0%)
-------------------
Total profit:  1.11 L
Total loss:  -23.09 L
-------------------
Total Booked + Current PnL: 16.44 L (13.29%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.19%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.28 C
Est FTT PnL: 88.36 L (63.29%)
Deployed:  1.24 C
Current:  1.40 C
CAGR/XIRR %: 7.4%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-43.66,61.0,H-MC,12.51,112858.0,12354.0,16748.0,-0.44,12.29,14.84,28.96,79.0,0.74,0.82,50.43,MH,ATH,METALS
77,TTKPRESTIG,770.00,93.17,36.0,M-SC,2.44,83198.0,-17579.0,17671.0,0.02,-17.44,21.24,0.09,245.0,-0.99,0.60,9.30,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,7.18,41.0,H-MC,3.36,184930.0,20260.0,23061.0,-2.44,12.30,12.47,26.31,89.0,0.88,1.34,21.27,X40N,NTT,AC
42,ITC,452.00,-38.77,46.0,H-LC,2.04,197874.0,-2264.0,23151.0,0.77,-1.13,11.70,10.44,4.0,-0.10,1.43,4.37,X40,NTT,FMCG
57,RELIANCE,1533.00,-14.41,43.0,H-LC,3.71,213907.0,3541.0,25241.0,0.53,1.68,11.80,13.68,37.0,0.14,1.55,18.48,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SURYODAY,240.0,68.86,77.0,H-SC,1.71,154530.0,-24541.0,90276.0,14.00,-13.70,58.42,36.71,135.0,-0.27,1.12,53.32,XR,NTT,BANKS
26,GREENPANEL,537.0,236.56,60.0,M-SC,1.40,150328.0,-32750.0,111724.0,11.61,-17.89,74.32,43.14,230.0,-0.29,1.09,41.40,XY24,NTT,MISC
64,SHALBY,327.0,1228.55,68.0,M-SC,14.20,180596.0,-547.0,46016.0,5.45,-0.30,25.48,25.10,235.0,-0.01,1.31,44.73,XY24,NTT,HEALTHCARE
73,TATAMOTORS,1065.0,-18.21,58.0,M-LC,2.26,258565.0,-17958.0,128041.0,4.72,-6.49,49.52,39.81,54.0,-0.14,1.87,23.92,XY24,NTT,AUTO
61,SAMMAANCAP,326.0,-186.36,80.0,M-SC,8.29,149940.0,720.0,143463.0,3.57,0.48,95.68,96.62,208.0,0.01,1.09,62.27,XY25,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.00,-48.24,23.0,M-SC,1.17,87502.0,-3996.0,82751.0,-5.35,-4.37,94.57,86.07,223.0,-0.05,0.63,27.07,XR,NTT,DURABLES
55,RAJOOENG,143.10,-46.62,19.0,H-SC,14.33,80970.0,-21530.0,62128.0,-5.32,-21.00,76.73,39.61,114.0,-0.35,0.59,0.00,AR,ATH,MISC
3,ALKYLAMINE,4546.37,-11.74,29.0,H-SC,10.69,82160.0,-18803.0,113332.0,-3.02,-18.62,137.94,93.63,148.0,-0.17,0.60,22.38,SR,ATH,CHEMICALS
81,VAIBHAVGBL,521.00,47.66,43.0,H-SC,5.80,127831.0,-54944.0,166014.0,-2.51,-30.06,129.87,60.77,125.0,-0.33,0.93,14.49,XR,NTT,JEWELLERY
14,BLUESTARCO,2080.00,7.18,41.0,H-MC,3.36,184930.0,20260.0,23061.0,-2.44,12.30,12.47,26.31,89.0,0.88,1.34,21.27,X40N,NTT,AC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.0,1228.55,68.0,M-SC,14.20,180596.0,-547.0,46016.0,5.45,-0.30,25.48,25.10,235.0,-0.01,1.31,44.73,XY24,NTT,HEALTHCARE
61,SAMMAANCAP,326.0,-186.36,80.0,M-SC,8.29,149940.0,720.0,143463.0,3.57,0.48,95.68,96.62,208.0,0.01,1.09,62.27,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,104.20,64.0,H-SC,11.43,129330.0,-13149.0,32669.0,3.34,-9.23,25.26,13.70,163.0,-0.40,0.94,52.64,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.61,55.0,H-SC,1.57,223407.0,-46260.0,82594.0,0.67,-17.15,36.97,13.47,138.0,-0.56,1.62,13.00,XY24,NTT,PAINTS
77,TTKPRESTIG,770.0,93.17,36.0,M-SC,2.44,83198.0,-17579.0,17671.0,0.02,-17.44,21.24,0.09,245.0,-0.99,0.60,9.30,OX40N,NTT,DURABLES
66,SIS,528.0,2100.52,44.0,H-SC,4.04,88804.0,-22228.0,45841.0,0.88,-20.02,51.62,21.26,156.0,-0.48,0.64,19.98,OX40N,NTT,MISC
18,CERA,9475.0,-23.15,35.0,H-SC,1.62,140254.0,-35649.0,77673.0,-0.98,-20.27,55.38,23.89,149.0,-0.46,1.02,21.13,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,45.95,57.0,M-MC,11.46,230369.0,5407.0,162249.0,-0.52,2.40,70.43,74.53,192.0,0.03,1.67,35.43,XY24,BTT,STEEL
32,HINDZINC,730.22,27.88,63.0,M-LC,3.83,208496.0,3420.0,109148.0,-0.65,1.67,52.35,54.89,52.0,0.03,1.51,26.16,X5K,ATH,METALS
14,BLUESTARCO,2080.00,7.18,41.0,H-MC,3.36,184930.0,20260.0,23061.0,-2.44,12.30,12.47,26.31,89.0,0.88,1.34,21.27,X40N,NTT,AC
84,VOLTAS,1530.00,-3.32,42.0,H-MC,2.84,202905.0,11163.0,26601.0,0.93,5.82,13.11,19.69,99.0,0.42,1.47,13.02,XY25,NTT,AC
17,CAMS,5211.76,-7.16,41.0,H-SC,1.82,105596.0,3592.0,40338.0,-0.76,3.52,38.20,43.06,122.0,0.09,0.77,21.24,X40N,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-43.66,61.0,H-MC,12.51,112858.0,12354.0,16748.0,-0.44,12.29,14.84,28.96,79.0,0.74,0.82,50.43,MH,ATH,METALS
11,BANKINDIA,190.00,-25.47,67.0,H-MC,9.44,191343.0,11535.0,100876.0,0.81,6.42,52.72,62.52,88.0,0.11,1.39,42.69,XR,NTT,BANKS
32,HINDZINC,730.22,27.88,63.0,M-LC,3.83,208496.0,3420.0,109148.0,-0.65,1.67,52.35,54.89,52.0,0.03,1.51,26.16,X5K,ATH,METALS
37,INDIAMART,4850.92,-55.20,25.0,H-SC,9.99,125149.0,1813.0,131945.0,-0.23,1.47,105.43,108.45,119.0,0.01,0.91,24.15,AR,ATH,MISC
86,WIPRO,420.00,-15.61,39.0,M-LC,5.79,148639.0,-2306.0,111762.0,0.15,-1.53,75.19,72.51,53.0,-0.02,1.08,4.40,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3465.66,-18.81,27.0,H-LC,5.82,204829.0,-46939.0,100141.0,-0.95,-18.64,48.89,21.13,27.0,-0.47,1.48,10.18,X40,ATH,PAINTS
74,TCS,4476.75,-31.58,29.0,H-LC,12.42,272149.0,-73807.0,148675.0,0.24,-21.33,54.63,21.64,1.0,-0.50,1.97,0.24,X40,ATH,IT
83,VBL,671.64,-21.40,32.0,H-LC,9.26,282889.0,-32953.0,145603.0,-0.07,-10.43,51.47,35.67,5.0,-0.23,2.05,1.98,X40N,ATH,FMCG
30,HCLTECH,1943.91,-4.95,33.0,H-LC,10.26,217414.0,-24510.0,87770.0,-0.02,-10.13,40.37,26.15,8.0,-0.28,1.58,2.74,X40,ATH,IT
44,JIOFIN,387.00,-5.85,34.0,H-LC,12.88,255936.0,-11429.0,76883.0,1.50,-4.27,30.04,24.48,48.0,-0.15,1.85,48.35,XY24,BTT,FINANCE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-8.09,40.0,H-LC,1.56,227678.0,-18473.0,97674.0,-1.91,-7.50,42.90,32.18,16.0,-0.19,1.65,25.83,X200,ATH,IT
65,SIEMENS,4671.5,-3.47,42.0,H-LC,1.99,155205.0,-30890.0,78363.0,-0.80,-16.60,50.49,25.51,15.0,-0.39,1.12,14.37,AR,ATH,ELECTRICAL
42,ITC,452.0,-38.77,46.0,H-LC,2.04,197874.0,-2264.0,23151.0,0.77,-1.13,11.70,10.44,4.0,-0.10,1.43,4.37,X40,NTT,FMCG
57,RELIANCE,1533.0,-14.41,43.0,H-LC,3.71,213907.0,3541.0,25241.0,0.53,1.68,11.80,13.68,37.0,0.14,1.55,18.48,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-22.87,40.0,H-LC,4.01,206254.0,1754.0,39663.0,1.18,0.86,19.23,20.25,10.0,0.04,1.49,9.67,X40N,BTT,PHARMA


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.58,29.0,H-LC,12.42,272149.0,-73807.0,148675.0,0.24,-21.33,54.63,21.64,1.0,-0.50,1.97,0.24,X40,ATH,IT
83,VBL,671.64,-21.40,32.0,H-LC,9.26,282889.0,-32953.0,145603.0,-0.07,-10.43,51.47,35.67,5.0,-0.23,2.05,1.98,X40N,ATH,FMCG
30,HCLTECH,1943.91,-4.95,33.0,H-LC,10.26,217414.0,-24510.0,87770.0,-0.02,-10.13,40.37,26.15,8.0,-0.28,1.58,2.74,X40,ATH,IT
40,INFY,2275.00,-21.39,37.0,H-LC,8.21,305698.0,-7190.0,178864.0,-0.46,-2.30,58.51,54.87,3.0,-0.04,2.21,4.20,X40,BTT,IT
42,ITC,452.00,-38.77,46.0,H-LC,2.04,197874.0,-2264.0,23151.0,0.77,-1.13,11.70,10.44,4.0,-0.10,1.43,4.37,X40,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-3.47,42.0,H-LC,1.99,155205.0,-30890.0,78363.0,-0.80,-16.60,50.49,25.51,15.0,-0.39,1.12,14.37,AR,ATH,ELECTRICAL
42,ITC,452.00,-38.77,46.0,H-LC,2.04,197874.0,-2264.0,23151.0,0.77,-1.13,11.70,10.44,4.0,-0.10,1.43,4.37,X40,NTT,FMCG
5,ASIANPAINT,3465.66,-18.81,27.0,H-LC,5.82,204829.0,-46939.0,100141.0,-0.95,-18.64,48.89,21.13,27.0,-0.47,1.48,10.18,X40,ATH,PAINTS
19,CIPLA,1795.00,-22.87,40.0,H-LC,4.01,206254.0,1754.0,39663.0,1.18,0.86,19.23,20.25,10.0,0.04,1.49,9.67,X40N,BTT,PHARMA
57,RELIANCE,1533.00,-14.41,43.0,H-LC,3.71,213907.0,3541.0,25241.0,0.53,1.68,11.80,13.68,37.0,0.14,1.55,18.48,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.58,29.0,H-LC,12.42,272149.0,-73807.0,148675.0,0.24,-21.33,54.63,21.64,1.0,-0.50,1.97,0.24,X40,ATH,IT
40,INFY,2275.00,-21.39,37.0,H-LC,8.21,305698.0,-7190.0,178864.0,-0.46,-2.30,58.51,54.87,3.0,-0.04,2.21,4.20,X40,BTT,IT
42,ITC,452.00,-38.77,46.0,H-LC,2.04,197874.0,-2264.0,23151.0,0.77,-1.13,11.70,10.44,4.0,-0.10,1.43,4.37,X40,NTT,FMCG
83,VBL,671.64,-21.40,32.0,H-LC,9.26,282889.0,-32953.0,145603.0,-0.07,-10.43,51.47,35.67,5.0,-0.23,2.05,1.98,X40N,ATH,FMCG
1,ABB,7934.00,-40.84,41.0,H-LC,7.73,247200.0,-14419.0,133636.0,-0.45,-5.51,54.06,45.57,7.0,-0.11,1.79,5.19,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,46.0,L-SC,40.40,80352.0,-33197.0,73249.0,3.51,-29.24,91.16,35.27,268.0,-0.45,0.58,96.66,XR,NTT,HOTELS
49,MASFIN,398.61,-20.96,35.0,H-SC,12.27,89775.0,-8205.0,29805.0,-1.12,-8.37,33.20,22.05,152.0,-0.28,0.65,30.49,XR,ATH,FINANCE
50,NATIONALUM,244.55,-43.66,61.0,H-MC,12.51,112858.0,12354.0,16748.0,-0.44,12.29,14.84,28.96,79.0,0.74,0.82,50.43,MH,ATH,METALS
78,UJJIVANSFB,60.00,104.20,64.0,H-SC,11.43,129330.0,-13149.0,32669.0,3.34,-9.23,25.26,13.70,163.0,-0.40,0.94,52.64,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-186.36,80.0,M-SC,8.29,149940.0,720.0,143463.0,3.57,0.48,95.68,96.62,208.0,0.01,1.09,62.27,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-186.36,80.0,M-SC,8.29,149940.0,720.0,143463.0,3.57,0.48,95.68,96.62,208.0,0.01,1.09,62.27,XY25,NTT,FINANCE
64,SHALBY,327.00,1228.55,68.0,M-SC,14.20,180596.0,-547.0,46016.0,5.45,-0.30,25.48,25.10,235.0,-0.01,1.31,44.73,XY24,NTT,HEALTHCARE
69,SURYODAY,240.00,68.86,77.0,H-SC,1.71,154530.0,-24541.0,90276.0,14.00,-13.70,58.42,36.71,135.0,-0.27,1.12,53.32,XR,NTT,BANKS
26,GREENPANEL,537.00,236.56,60.0,M-SC,1.40,150328.0,-32750.0,111724.0,11.61,-17.89,74.32,43.14,230.0,-0.29,1.09,41.40,XY24,NTT,MISC
50,NATIONALUM,244.55,-43.66,61.0,H-MC,12.51,112858.0,12354.0,16748.0,-0.44,12.29,14.84,28.96,79.0,0.74,0.82,50.43,MH,ATH,METALS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.15
1,25,43.43
2,50,74.93


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.20
LC    32.30
MC    23.52
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.45
X40      14.56
X40N     11.32
XR       11.17
XY25     10.57
AR        8.13
OX40N     7.85
X200      1.65
MH        1.63
X5K       1.51
SR        1.18
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.23
H-LC    25.65
H-MC    20.57
M-SC    13.52
M-LC     5.60
M-MC     2.64
L-SC     1.45
L-LC     1.05
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.95,-6.71,41.92
IT,12.55,-21.43,87.64
FINANCE,10.47,-16.73,69.75
MISC,7.04,-18.35,84.15
BANKS,6.49,-11.70,70.46
PAINTS,5.58,-19.94,37.81
ELECTRICAL,5.37,-10.89,49.79
INSURANCE,3.77,-6.23,45.94
AC,3.51,0.16,16.55


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3177968.0,22
XR,1369105.0,14
AR,1209578.0,9
X40,836024.0,10
X40N,643066.0,9
OX40N,563226.0,10
XY25,481129.0,7
SR,259209.0,2
X5K,109148.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3764762.0,29
M-SC,1472696.0,17
H-LC,1277581.0,15
H-MC,1269855.0,15
M-LC,377182.0,4
M-MC,312992.0,2
L-SC,260929.0,3
L-MC,60321.0,1
L-LC,39548.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1237772.0      6
           AR         828120.0      5
M-SC       XY24       811081.0      7
H-SC       XR         792443.0      7
H-MC       XY24       582968.0      4
H-LC       X40        538601.0      5
H-SC       X40N       316416.0      4
M-MC       XY24       312992.0      2
H-SC       SR         259209.0      2
           OX40N      257811.0      4
H-LC       X40N       236065.0      3
H-MC       X40        221938.0      4
H-LC       AR         211999.0      2
H-MC       XY25       181759.0      2
L-SC       XR         172048.0      2
M-SC       AR         169459.0      2
M-LC       XY24       156272.0      2
M-SC       XY25       143463.0      1
           OX40N      141553.0      4
           XR         131655.0      2
H-LC       XY25       116359.0      3
M-LC       XR         111762.0      1
           X5K        109148.0      1
H-MC       XR         100876.0      1
H-LC       X200        97674.0      1
H-MC       X40N        90585.0      2
L-SC       OX40N       88881.0      1
H-LC       XY24        76883.0      1
M-SC       X40         75485.0      1
H-MC       OX40N       74981.0      1
H-SC       MH          72991.0      1
L-MC       XR          60321.0      1
L-LC       XY25        39548.0      1
H-MC       MH          16748.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 24.0 seconds
